In [2]:
NEUROGLANCER_ROOT = '/media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/'


import socket
import neuroglancer
import subprocess
import numpy as np
from neuroglancer.server import global_server_args

class Neuroglancer_Viewer:
    def __init__(self, stack='MD585'):
        self.stack = stack
        self.local_volume_fp_root = './'
        
        neuroglancer.set_server_bind_address('0.0.0.0')
        global_server_args['bind_port'] = 80
        
        # Create viewer
        self.viewer = neuroglancer.Viewer()
        
        # Get the IP address
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip_name = s.getsockname()[0]
        s.close()

        # Removes the following symbols: ', ", [, ]
        self.url = str( 'http://' + ip_name + ':' + self.viewer.get_viewer_url().split(':')[2] )##Remote URL
        
    def set_local_volume_fp(self, fp):
        if fp[len(fp)-1]!='/':
            fp = fp+'/'
        self.local_volume_fp_root = fp
        
    def download_volumes(self):
        s3_root_fp = 's3://test-bucket-sid/alex_neuroglancer_volumes/'+self.stack+'/human_annotations_5um/'
        local_volume_fp = self.local_volume_fp_root+self.stack+'/human_annotations_5um/'
        
        command_list = ['aws','s3','cp','--recursive', s3_root_fp, local_volume_fp]
        subprocess.call( command_list )
    
    def add_stack(self):
        if self.stack == 'MD585':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD585_fullres')
        elif self.stack == 'MD589':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD589_fullres')
        elif self.stack == 'MD594':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD594_fullres')
            
    def add_volume(self, colored=True):
        volume_fp = self.local_volume_fp_root+self.stack+'/human_annotation/solid_volume_5um/'
        if colored:
            volume_fn = 'volume_colored.npy'
            color_segments=[]
            for i in range(1,50):
                color_segments.append(i)
        else:
            volume_fn = 'volume.npy'
        
        xy_ng_resolution_um = 5
        volume_data = np.load( volume_fp+volume_fn )
        
        with self.viewer.txn() as s:
            s.layers[ self.stack+"_Annotations" ] = neuroglancer.SegmentationLayer(
                source = neuroglancer.LocalVolume(
                    data=volume_data, # Z,Y,X
                    voxel_size=[ xy_ng_resolution_um*1000, xy_ng_resolution_um*1000, 20000], # X Y Z
                    voxel_offset = [ 0, 0, 0] # X Y Z
                ),
                segments = color_segments
            )
            
    def reset_orientation(self):
        with self.viewer.txn() as s:
            # Resets X/Y/Z plane orientation
            s.navigation.pose.orientation = [0,0,0,1]
            # Zooms out 
            s.navigation.zoomFactor = 5000 # 5000 If xy, 10000 If 4panel
            # Resets 3D Viewer Orientation
            s.perspectiveOrientation = [0,0,0,1]
            # Zooms out
            s.perspectiveZoom = 75000

            s.layout = 'xy' # '3d'/'4panel'/'xy'
            
    def stop(self):
        neuroglancer.stop()

In [3]:
viewers = []
print 'MD585'
viewers.append( Neuroglancer_Viewer(stack='MD585') )
viewers[0].add_stack()
viewers[0].set_local_volume_fp( fp=NEUROGLANCER_ROOT )
viewers[0].add_volume()
viewers[0].reset_orientation()
print viewers[0].url

print 'MD589'
viewers.append( Neuroglancer_Viewer(stack='MD589') )
viewers[1].add_stack()
viewers[1].set_local_volume_fp( fp=NEUROGLANCER_ROOT )
viewers[1].add_volume()
viewers[1].reset_orientation()
print viewers[1].url

print 'MD594'
viewers.append( Neuroglancer_Viewer(stack='MD594') )
viewers[2].add_stack()
viewers[2].set_local_volume_fp( fp=NEUROGLANCER_ROOT )
viewers[2].add_volume()
viewers[2].reset_orientation()
print viewers[2].url

MD585
http://132.239.73.85:45091/v/da0ac5f920d75757ac0e9de870691da81af080d9/
MD589
http://132.239.73.85:45091/v/881e752e45c4f75ecf44fbad37254f24b18408c7/
MD594
http://132.239.73.85:45091/v/f6c737bb33e47563b7a4f1dbd13a512d4e924504/


In [5]:
viewers[0].stop()
del viewers[0]
viewers[0].stop()
del viewers[0]
viewers[0].stop()
del viewers[0]

In [3]:
new_viewer = Neuroglancer_Viewer(stack='MD594') # 8 off
new_viewer.add_stack()
new_viewer.reset_orientation()

print new_viewer.url

http://132.239.73.85:42849/v/b6486a96bca246b911bfeb7a6647331877432ce7/


In [3]:
new_viewer = Neuroglancer_Viewer(stack='MD589') # 8 off
new_viewer.add_stack()
new_viewer.reset_orientation()

print new_viewer.url

http://132.239.73.85:42211/v/e61617fb1f4f04f4e8e9310c73b1deeb0ce4dd25/


In [3]:
new_viewer = Neuroglancer_Viewer(stack='MD585')
new_viewer.add_stack()

new_viewer.set_local_volume_fp( fp=NEUROGLANCER_ROOT )
new_viewer.add_volume()

new_viewer.reset_orientation()

print new_viewer.url

http://132.239.73.85:32961/v/a78262516d1329bc39c62a444903a6fdeb007c50/


In [4]:
new_viewer.set_local_volume_fp( fp=NEUROGLANCER_ROOT )
new_viewer.add_volume()
new_viewer.reset_orientation()

In [17]:
new_viewer.stop()
del new_viewer

In [57]:
import subprocess
s3_root_fp = 's3://test-bucket-sid/alex_neuroglancer_volumes/'+'MD585'+'/human_annotations_5um'
        
command_list = ['aws','s3','cp','--recursive',s3_root_fp, '/home/alexn/Desktop/test_ng/']
subprocess.call( command_list )

1

In [41]:
! rm -r /home/alexn/test_ng

In [1]:
NEUROGLANCER_ROOT = '/media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/'


import socket
import neuroglancer
import subprocess
import numpy as np
from neuroglancer.server import global_server_args

class Neuroglancer_Viewer:
    def __init__(self):
        self.local_volume_fp_root = './'
        
        neuroglancer.set_server_bind_address('0.0.0.0')
        global_server_args['bind_port'] = 80
        
        # Create viewer
        self.viewer = neuroglancer.Viewer()
        
        # Get the IP address
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip_name = s.getsockname()[0]
        s.close()

        # Removes the following symbols: ', ", [, ]
        self.url = str( 'http://' + ip_name + ':' + self.viewer.get_viewer_url().split(':')[2] )##Remote URL
        
    def set_local_volume_fp(self, fp):
        if fp[len(fp)-1]!='/':
            fp = fp+'/'
        self.local_volume_fp_root = fp
        
    def download_volumes(self):
        s3_root_fp = 's3://test-bucket-sid/alex_neuroglancer_volumes/'+self.stack+'/human_annotations_5um/'
        local_volume_fp = self.local_volume_fp_root+self.stack+'/human_annotations_5um/'
        
        command_list = ['aws','s3','cp','--recursive', s3_root_fp, local_volume_fp]
        subprocess.call( command_list )
    
    def add_fp(self, fp):
        with self.viewer.txn() as s:
            s.layers[ fp+'_image'] = neuroglancer.ImageLayer(\
                source='precomputed:'+fp)
        stack='p'
        if stack == 'MD589':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD589_fullres')
        elif stack == 'MD594':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD594_fullres')
            
    def add_volume(self, colored=True):
        volume_fp = self.local_volume_fp_root+self.stack+'/human_annotation/solid_volume_5um/'
        if colored:
            volume_fn = 'volume_colored.npy'
            color_segments=[]
            for i in range(1,50):
                color_segments.append(i)
        else:
            volume_fn = 'volume.npy'
        
        xy_ng_resolution_um = 5
        volume_data = np.load( volume_fp+volume_fn )
        
        with self.viewer.txn() as s:
            s.layers[ self.stack+"_Annotations" ] = neuroglancer.SegmentationLayer(
                source = neuroglancer.LocalVolume(
                    data=volume_data, # Z,Y,X
                    voxel_size=[ xy_ng_resolution_um*1000, xy_ng_resolution_um*1000, 20000], # X Y Z
                    voxel_offset = [ 0, 0, 0] # X Y Z
                ),
                segments = color_segments
            )
            
    def reset_orientation(self):
        with self.viewer.txn() as s:
            # Resets X/Y/Z plane orientation
            s.navigation.pose.orientation = [0,0,0,1]
            # Zooms out 
            s.navigation.zoomFactor = 5000 # 5000 If xy, 10000 If 4panel
            # Resets 3D Viewer Orientation
            s.perspectiveOrientation = [0,0,0,1]
            # Zooms out
            s.perspectiveZoom = 75000

            s.layout = 'xy' # '3d'/'4panel'/'xy'
            
    def stop(self):
        neuroglancer.stop()

In [2]:
new_viewer = Neuroglancer_Viewer() # 8 off


print( new_viewer.url )

http://132.239.73.85:38325/v/97b5ad4fee0caf37fdb5aa8bd5877e2c59dfafd1/


In [3]:
new_viewer.add_fp('/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output')
new_viewer.reset_orientation()

AttributeError: 'Neuroglancer_Viewer' object has no attribute 'stack'

In [ ]:
new_viewer.add_fp('/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output')
new_viewer.reset_orientation()

In [7]:
NEUROGLANCER_ROOT = '/media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/'


import socket
import neuroglancer
import subprocess
import numpy as np
from neuroglancer.server import global_server_args

neuroglancer.set_server_bind_address('0.0.0.0')
viewer = neuroglancer.Viewer()
print(viewer)

ip_name = !hostname -I
print('http://' + ip_name[0].strip() + ':' + viewer.get_viewer_url().split(':')[2]) ##Remote URL

ip_name_str = str(ip_name).replace('\'','').replace('\"','').replace('[','').replace(']','')
real_ip = ip_name_str.split(' ')[0]
print('http://' + real_ip + ':' + viewer.get_viewer_url().split(':')[2]) ##Remote URL

viewer # port 41989, IP 132.239.73.85

# Sets 'Image' layer to be prep2 images from S3 of <stack>
with viewer.txn() as s:
    s.layers['primary'] = neuroglancer.ImageLayer(source='precomputed://https://neuroglancer-data.s3.amazonaws.com/Tests/primary',
        shader="""
                void main() {
                    emitRGB(vec3( toNormalized(getDataValue(0)), toNormalized(getDataValue(1)), 0));
                }
                """)
    s.layers['secondary'] = neuroglancer.ImageLayer(source='precomputed://https://neuroglancer-data.s3.amazonaws.com/Tests/secondary',
        shader="""
                void main() {
                    emitRGB(vec3( toNormalized(getDataValue(0)), 0, toNormalized(getDataValue(2))));
                }
                """)

http://ratto.dk.ucsd.edu:41667/v/47ae6d7678226fcf3b0b6e9e3d58eb3519b4ac66/
http://132.239.73.85 172.17.0.1 172.18.0.1:41667/v/47ae6d7678226fcf3b0b6e9e3d58eb3519b4ac66/
http://132.239.73.85:41667/v/47ae6d7678226fcf3b0b6e9e3d58eb3519b4ac66/


In [104]:
# Sets 'Image' layer to be prep2 images from S3 of <stack>
with viewer.txn() as s:
    s.layers['primary'] = neuroglancer.ImageLayer(source='precomputed://https://neuroglancer-data.s3.amazonaws.com/Tests/primary',
        shader="""
                void main() {
                  emitRGB(vec3( toNormalized(getDataValue(0)), toNormalized(getDataValue(1)), toNormalized(getDataValue(2))));
                }
                """)
    s.layers['secondary'] = neuroglancer.ImageLayer(source='precomputed://https://neuroglancer-data.s3.amazonaws.com/Tests/secondary',
        shader="""
void main() {
    emitRGB(vec3(toNormalized(getDataValue(0)),
        toNormalized(getDataValue(1)),
        toNormalized(getDataValue(2))));
}
                """)
    
#     # Resets X/Y/Z plane orientation
#     s.navigation.pose.orientation = [0,0,0,1]
#     # Zooms out 
# #     s.navigation.zoomFactor = 10000 # If 4panel
#     s.navigation.zoomFactor = 5000 # If xy
    
#     # Resets 3D Viewer Orientation
#     s.perspectiveOrientation = [0,0,0,1]
#     # Zooms out
#     s.perspectiveZoom = 75000
    
#     s.layout = 'xy' # '3d'/'4panel'/'xy'

In [88]:
# Sets 'Image' layer to be prep2 images from S3 of <stack>
with viewer.txn() as s:
    s.layers['primary'] = neuroglancer.ImageLayer(source='precomputed://https://neuroglancer-data.s3.amazonaws.com/Tests/primary')
    s.layers['secondary'] = neuroglancer.ImageLayer(source='precomputed://https://neuroglancer-data.s3.amazonaws.com/Tests/secondary')
    
    # Resets X/Y/Z plane orientation
    s.navigation.pose.orientation = [0,0,0,1]
    # Zooms out 
#     s.navigation.zoomFactor = 10000 # If 4panel
    s.navigation.zoomFactor = 5000 # If xy
    
    # Resets 3D Viewer Orientation
    s.perspectiveOrientation = [0,0,0,1]
    # Zooms out
    s.perspectiveZoom = 75000
    
    s.layout = 'xy' # '3d'/'4panel'/'xy'

In [ ]:
# Sets 'Image' layer to be prep2 images from S3 of <stack>
with viewer.txn() as s:
    s.layers['image'] = neuroglancer.ImageLayer(source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/'+stack+'_fullres')
    
    # Resets X/Y/Z plane orientation
    s.navigation.pose.orientation = [0,0,0,1]
    # Zooms out 
#     s.navigation.zoomFactor = 10000 # If 4panel
    s.navigation.zoomFactor = 5000 # If xy
    
    # Resets 3D Viewer Orientation
    s.perspectiveOrientation = [0,0,0,1]
    # Zooms out
    s.perspectiveZoom = 75000
    
    s.layout = 'xy' # '3d'/'4panel'/'xy'

In [ ]:
with viewer.txn() as s:
    s.voxel_size = [10, 10, 10]
    s.layers.append(
        name='a',
        layer=neuroglancer.LocalVolume(
            data=a,
            # offset is in nm, not voxels
            offset=(200, 300, 150),
            voxel_size=s.voxel_size,
        ),
        shader="""
                void main() {
                  emitRGB(vec3(toNormalized(getDataValue(0)),
                               toNormalized(getDataValue(1)),
                               toNormalized(getDataValue(2))));
                }
                """)
    s.layers.append(
        name='b', layer=neuroglancer.LocalVolume(
            data=b,
            voxel_size=s.voxel_size,
        ))